In [4]:
import sys
sys.path.append("/Users/Bing/Documents/DS/DrivenData/Pover-T/Scripts/") # need to add path to the parent folder where CV.py is
from PoverTHelperTools import *
import os 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('white')

from sklearn.linear_model import LogisticRegression

In [5]:
hhold_a_train, hhold_b_train, hhold_c_train = load_hhold_train()
hhold_a_test, hhold_b_test, hhold_c_test = load_hhold_test()

# indiv_a_train, indiv_b_train, indiv_c_train = load_indiv_train()
# indiv_a_test, indiv_b_test, indiv_c_test = load_indiv_test()

In [13]:
def train_LR(df, df_test):
    X = pre_process_data(df.drop(['poor', 'country'], axis = 1))
    y = np.ravel(df['poor']).astype(int)
    
    model = LogisticRegression()
    model.fit(X,y)

    a_test = pre_process_data(df_test, enforce_cols=X.columns)
    
    preds = model.predict_proba(a_test)
    return model, preds

In [17]:
# aX_train = pre_process_data(hhold_a_train.drop(['poor', 'country'], axis = 1))
# ay_train = np.ravel(hhold_a_train['poor']).astype(int)

a_LR_model, a_preds = train_LR(hhold_a_train, hhold_a_test)
b_LR_model, b_preds = train_LR(hhold_b_train, hhold_b_test)
c_LR_model, c_preds = train_LR(hhold_c_train, hhold_c_test)

Input shape:	(8203, 343)
After standardization (8203, 343)
After converting categoricals:	(8203, 858)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8203 entries, 46107 to 39832
Columns: 858 entries, nEsgxvAq to AlDbXTlZ_cecIq
dtypes: float64(4), uint8(854)
memory usage: 7.0 MB
Input shape:	(4041, 344)
After standardization (4041, 344)
After converting categoricals:	(4041, 851)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4041 entries, 418 to 43344
Columns: 851 entries, nEsgxvAq to country_A
dtypes: float64(4), uint8(847)
memory usage: 3.4 MB
Input shape:	(3255, 440)
After standardization (3255, 440)
After converting categoricals:	(3255, 1431)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255 entries, 57071 to 4923
Columns: 1431 entries, wJthinfa to ChbSWYhO_zmaYX
dtypes: float64(23), uint8(1408)
memory usage: 5.0 MB
Input shape:	(1604, 441)
After standardization (1604, 441)
After converting categoricals:	(1604, 1419)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1604 

In [18]:
def make_country_sub(preds, test_feat, country):

    
    # just get probabilities for p = 1 with preds[:,1]
    country_sub = pd.DataFrame(data = preds[:,1],
                              columns = ['poor'],
                              index = test_feat.index)
    
    # add country code to join later on
    country_sub['country'] = country
    return country_sub[['country', 'poor']]

In [19]:
# lgbm subs
a_sub = make_country_sub(a_preds, hhold_a_test, 'A')
b_sub = make_country_sub(b_preds, hhold_b_test, 'B')
c_sub = make_country_sub(c_preds, hhold_c_test, 'C')
sub = pd.concat([a_sub, b_sub, c_sub])
sub.to_csv('../Submissions/LR_v1.csv')